In [1]:
from data_prep import *
from charts import *
from video_analysis import *
from team_sheets import *

# Load data
game_df = team_sheets()
players_df = players(game_df)
players_agg_df = players_agg(players_df)
lineouts_df = lineouts()
set_piece_df = set_piece_results()
analysis = game_stats()
 
# # Save data
# game_df.to_csv('data/game.csv', index=False)
# players_df.to_csv('data/players.csv', index=False)
# players_agg_df.to_csv('data/players_agg.csv', index=False)
# lineouts_df.to_csv('data/lineouts.csv', index=False)
# set_piece_df.to_csv('data/set_piece.csv', index=False)
# analysis.to_csv('data/analysis.csv', index=False)
# update_season_summaries(game_df, seasons=["2024/25"])

# # One-off charts (only source data needs updating)
# captains_chart(file='Charts/captains.html')
# results_chart(file='Charts/results.html')
# plot_games_by_player(file='Charts/appearances.html')
# plot_starts_by_position(file='Charts/positions.html')
# card_chart(file='Charts/cards.html')
# points_scorers_chart(file='Charts/points.html')
# team_sheets_chart(file='Charts/team-sheets.html')
# set_piece_h2h_chart(file='Charts/set-piece.html')
squad_continuity_chart(file='Charts/continuity.html')

# # Self-contained charts (chart needs updating)
# game_stats_charts(analysis, file='Charts/video_analysis.html')
# lineout_success(types=types, file='Charts/lineouts.html')

Updated Charts/continuity.html


alt.HConcatChart(...)

In [2]:
game_df.to_json('data/game.json', orient='records')

In [ ]:
from IPython.core.display import display, HTML
import pandas as pd

# Calculate summary statistics
def generate_season_summary(df, season):
    season_data = df[df["Season"] == season]
    summary = season_data.groupby("Squad").agg(
        Played=("GameID", "count"),
        Won=("Result", lambda x: (x == "W").sum()),
        Lost=("Result", lambda x: (x == "L").sum()),
        Avg_PF=("PF", "mean"),
        Avg_PA=("PA", "mean"),
    ).reset_index()
    
    # Convert to dictionary for easy JS manipulation
    summary_dict = summary.set_index("Squad").T.to_dict()
    
    return summary_dict

# Generate dropdown options
seasons = game_df["Season"].unique()
dropdown_options = "".join([f'<option value="{s}">{s}</option>' for s in seasons[::-1]])

# Generate initial table for the first season
initial_season = seasons[-1]
initial_summary = generate_season_summary(game_df, initial_season)

# HTML Output
# Inject into existing index.html
with open("index.html", "r", encoding="utf-8") as f:
    index_html = f.read()


soup = BeautifulSoup(index_html, "html.parser")

# Update dropdown options
select = soup.find("select", id="seasonSelect")
select.clear()
for season in seasons[::-1]:
    option = soup.new_tag("option")
    option["value"] = season
    option.string = season
    select.append(option)

# Update table with initial summary
table = soup.find("table")
for squad, data in initial_summary.items():
    row = table.find("tr", id=f"row{squad[0]}")
    row.find("td", id=f"played{squad[0]}").string = str(int(data["Played"]))
    row.find("td", id=f"won{squad[0]}").string = str(int(data["Won"]))
    row.find("td", id=f"lost{squad[0]}").string = str(int(data["Lost"]))
    row.find("td", id=f"pf{squad[0]}").string = f"{data['Avg_PF']:.1f}"
    row.find("td", id=f"pa{squad[0]}").string = f"{data['Avg_PA']:.1f}"

# Save updated index.html
with open("index.html", "w", encoding="utf-8") as f:
    f.write(str(soup))


# html_output = f"""
# <!DOCTYPE html>
# <html lang="en">
# <head>
#     <meta charset="UTF-8">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
#     <title>Rugby Season Summary</title>
#     <script>
#         const summaries = {initial_summary};

#         function updateTable() {{
#             const selectedSeason = document.getElementById("seasonSelect").value;
#             fetch("data/" + selectedSeason + ".json")
#                 .then(response => response.json())
#                 .then(data => {{
#                     document.getElementById("played1").innerText = data["1st XV"].Played;
#                     document.getElementById("won1").innerText = data["1st XV"].Won;
#                     document.getElementById("lost1").innerText = data["1st XV"].Lost;
#                     document.getElementById("pf1").innerText = data["1st XV"].Avg_PF.toFixed(1);
#                     document.getElementById("pa1").innerText = data["1st XV"].Avg_PA.toFixed(1);
                    
#                     document.getElementById("played2").innerText = data["2nd XV"].Played;
#                     document.getElementById("won2").innerText = data["2nd XV"].Won;
#                     document.getElementById("lost2").innerText = data["2nd XV"].Lost;
#                     document.getElementById("pf2").innerText = data["2nd XV"].Avg_PF.toFixed(1);
#                     document.getElementById("pa2").innerText = data["2nd XV"].Avg_PA.toFixed(1);
#                 }});
#         }}
#     </script>
# </head>
# <body class="container mt-4">
#     <h2 class="text-center">Rugby Season Summary</h2>
#     <div class="mb-3">
#         <label for="seasonSelect" class="form-label">Select Season:</label>
#         <select id="seasonSelect" class="form-select" onchange="updateTable()">
#             {dropdown_options}
#         </select>
#     </div>
    
#     <div class="table-responsive">
#         <table class="table table-bordered text-center align-middle">
#             <thead class="table-dark">
#                 <tr>
#                     <th>Squad</th>
#                     <th>Played</th>
#                     <th>Won</th>
#                     <th>Lost</th>
#                     <th>Avg PF</th>
#                     <th>Avg PA</th>
#                 </tr>
#             </thead>
#             <tbody>
#                 <tr>
#                     <td class="table-primary">1st XV</td>
#                     <td id="played1">{initial_summary["1st XV"]["Played"]}</td>
#                     <td id="won1">{initial_summary["1st XV"]["Won"]}</td>
#                     <td id="lost1">{initial_summary["1st XV"]["Lost"]}</td>
#                     <td id="pf1">{initial_summary["1st XV"]["Avg_PF"]:.1f}</td>
#                     <td id="pa1">{initial_summary["1st XV"]["Avg_PA"]:.1f}</td>
#                 </tr>
#                 <tr>
#                     <td class="table-secondary">2nd XV</td>
#                     <td id="played2">{initial_summary["2nd XV"]["Played"]}</td>
#                     <td id="won2">{initial_summary["2nd XV"]["Won"]}</td>
#                     <td id="lost2">{initial_summary["2nd XV"]["Lost"]}</td>
#                     <td id="pf2">{initial_summary["2nd XV"]["Avg_PF"]:.1f}</td>
#                     <td id="pa2">{initial_summary["2nd XV"]["Avg_PA"]:.1f}</td>
#                 </tr>
#             </tbody>
#         </table>
#     </div>
# </body>
# </html>
# """

# display(HTML(html_output))

# # Save to file
# with open("season_summary.html", "w", encoding="utf-8") as f:
#     f.write(html_output)

# Save JSON summaries for JavaScript fetching
import json
for season in seasons:
    summary = generate_season_summary(game_df, season)
    with open(f"data/{season.replace('/','-')}.json", "w") as f:
        json.dump(summary, f, indent=4)


/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_91865/1267223303.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


NameError: name 'game_df' is not defined

# Individual Player Stats

In [ ]:
def player_data(p):
    pitchero = pitchero_df[pitchero_df["Player"]==p]
    pdf = players_df[players_df["Player"]==p]
    p_agg = players_agg_df[players_agg_df["Player"]==p]
    
    return {
        "pitchero": pitchero,
        "pdf": pdf,
        "p_agg": p_agg
    }

a,b,c = player_data("Sam Lindsay-McCall").values()    

In [ ]:
p = "Dan Billin"

def squad_pie(p):
    base = (
        alt.Chart(players_agg_df).encode(
            theta=alt.Theta("sum(TotalGames)").stack(True),
            color=alt.Color("Squad:N", scale=squad_scale, legend=alt.Legend(title=None, labelExpr="datum.label + ' XV'"))
        )
        .transform_filter(f"datum.Player === '{p}'")
        .transform_calculate(label="datum.Squad + ' XV'")
    )

    pie = base.mark_arc(outerRadius=120, opacity=0.8)
    text1 = base.mark_text(radius=75, size=36).encode(
        theta=alt.Theta("sum(TotalGames)", stack=True),
        text=alt.Text("sum(TotalGames)"), 
        detail="Squad:N",
        color=alt.value("white")
    )
    text2 = base.mark_text(radius=150, size=24).encode(
        theta=alt.Theta("sum(TotalGames)", stack=True),
        text=alt.Text("label:N"),
        detail="Squad:N",
    )

    return pie + text1 + text2

position_order = ["Prop", "Hooker", "Second Row", "Back Row", "Scrum Half", "Fly Half", "Centre", "Back Three"]
position_color = ["#202947", "#146f14", "#981515", "#b03030"]

def position_pie(p):
    base = (
        alt.Chart(players_df)
        .transform_calculate(posi=f"indexof({position_order}, datum.Position)")
        .encode(
            theta=alt.Theta("count()").stack(True),
            color=alt.Color(
                "Position:N"
                legend=alt.Legend(title=None, orient="bottom", offset=40), 
                scale=alt.Scale()
            )
        )
        .transform_filter(f"datum.Player === '{p}' & isValid(datum.Position)")
    )

    pie = base.mark_arc(outerRadius=120)
    text = base.mark_text(radius=75, size=36).encode(
        theta=alt.Theta("count()", stack=True),
        text=alt.Text("count()"), 
        color=alt.value("white"),
        detail="Position:N"
    )

    return (pie + text).transform_filter(f"datum.Player === '{p}'")

def games(p):
    bar = (
        alt.Chart(players_agg_df).encode(
            x=alt.X("Date:T", title="Date"),
            y=alt.Y("count()", title="Games Played"),
            color=alt.Color("Position:N", scale=position_scale)
        )
        .transform_filter(f"datum.Player === '{p}'")
        .mark_bar()
    )

position_pie(p)
# squad_pie(p)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2585547723.py, line 38)

In [ ]:
b

,Squad,Season,Competition,GameType,Opposition,Home/Away,PF,PA,Result,Captain,VC1,VC2,Number,Player,Position,PositionType
422,1st,2021/22,Friendly,Friendly,Metropolitan Police,H,29,28,W,Jack Andrews,<NA>,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
423,1st,2021/22,Friendly,Friendly,London Irish,A,10,33,L,Jack Andrews,Sam Lindsay-McCall,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
424,1st,2021/22,Friendly,Friendly,Horsham,A,26,56,L,Jack Andrews,James Funnell,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
425,1st,2021/22,Friendly,Friendly,Purley John Fisher,H,19,33,L,Jack Andrews,James Funnell,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
426,1st,2021/22,Sussex 1,League,Eastbourne,H,47,0,W,Jack Andrews,<NA>,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,1st,2023/24,Counties 1 Surrey/Sussex,League,Eastbourne,A,24,26,L,Ryland Thomas,Dan Poulton,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2313,1st,2024/25,Counties 1 Surrey/Sussex,League,Eastbourne,A,36,33,W,Ryland Thomas,Chris May-Miller,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2314,1st,2024/25,Counties 1 Surrey/Sussex,League,Trinity,H,14,48,L,Ryland Thomas,<NA>,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2315,1st,2024/25,Counties 1 Surrey/Sussex,League,Old Rutlishians,H,19,23,L,Ryland Thomas,<NA>,<NA>,17,Sam Lindsay-McCall,NaN,Bench


# Team Sheets

In [ ]:
def team_sheet_chart(
        squad=1, 
        names=None, 
        captain=None, 
        vc=None, 
        opposition=None, 
        home=True, 
        competition="Counties 1 Sussex",
        season="2023/24"
    ):

    if names is None:
        df = team_sheets()    

        # Last row as dict
        team = df.iloc[-1].to_dict()


        label = f'{"1st" if squad==1 else "2nd"} XV vs {team["Opposition"]}({team["Home/Away"]})'
        captain = team["Captain"]
        vc = team["VC1"]
        season = team["Season"]
        competition = team["Competition"]

        # Keep keys that can be converted to integers
        team = {int(k): v for k, v in team.items() if k.isnumeric() and v}

        # Convert team to dataframe with Number and Player columns
        team = pd.DataFrame(team.items(), columns=["Number", "Player"])

    else:
        label = f'{"1st" if squad==1 else "2nd"} XV vs {opposition} ({"H" if home else "A"})'

        # Convert names to Player column of a dataframe with Number column (1-len(names))
        team = pd.DataFrame({"Player": names, "Number": range(1, len(names)+1)})

    coords = pd.DataFrame([
                {"n": 1, "x": 10, "y": 81},
                {"n": 2, "x": 25, "y": 81},
                {"n": 3, "x": 40, "y": 81},
                {"n": 4, "x": 18, "y": 69},
                {"n": 5, "x": 32, "y": 69},
                {"n": 6, "x": 6, "y": 61},
                {"n": 7, "x": 44, "y": 61},
                {"n": 8, "x": 25, "y": 56},
                {"n": 9, "x": 20, "y": 42},
                {"n": 10, "x": 38, "y": 36},
                {"n": 11, "x": 8, "y": 18},
                {"n": 12, "x": 56, "y": 30},
                {"n": 13, "x": 74, "y": 24},
                {"n": 14, "x": 92, "y": 18},
                {"n": 15, "x": 50, "y": 10},
                {"n": 16, "x": 80, "y": 82},
                {"n": 17, "x": 80, "y": 74},
                {"n": 18, "x": 80, "y": 66},
                {"n": 19, "x": 80, "y": 58},
                {"n": 20, "x": 80, "y": 50},
                {"n": 21, "x": 80, "y": 42},
                {"n": 22, "x": 80, "y": 34},
                {"n": 23, "x": 80, "y": 26},
            ])
    team = team.merge(coords, left_on="Number", right_on="n", how="inner").drop(columns="n")

    # Add captain (C) and vice captain (VC) else None
    team["Captain"] = team["Player"].apply(lambda x: "C" if x == captain else "VC" if x == vc else None)

    team["Player"] = team["Player"].str.split(" ")

    team.to_dict(orient="records")

    with open("team-sheet-lineup.json") as f:
        chart = json.load(f)
    chart["data"]["values"] = team.to_dict(orient="records")
    chart["title"]["text"] = label
    chart["title"]["subtitle"] = f"{season} - {competition}"

    n_replacements = len(team) - 15
    
    y = 126 + (n_replacements * 64)
    chart["layer"][0]["mark"]["y2"] = y
    # return chart
    return alt.Chart.from_dict(chart)

team_sheet_chart()

alt.LayerChart(...)

In [ ]:
pitchero_df[pitchero_df["Player"]=="Sam Lindsay-McCall"]

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens,Cards
9,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pitchero_df[pitchero_df["Player_join"]=="S Lindsay"].sort_values(["Season","Squad"])

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens
464,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
